<h1> 2. (Pret)procesiranje podataka </h1>

U ovoj bilježnici radimo na čišćenju podataka te definiciji novih značajki. Input je skup za treniranje dan na natjecanju, a output je skup spreman za transformacije kao što su enkodiranje i normalizacija.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
## DIO POTREBAN ZA BRISANJE UPOZORENJA (kako bismo sačuvali anonimnost)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_hdf('./data/raw/training_dataset.h5', 'df')

In [3]:
### BRISANJE PARTIJA S VIŠE KLIJENATA
old_shape = df.shape
gb = df.groupby('OZNAKA_PARTIJE')['KLIJENT_ID'].nunique() # baci broj jedinstvenih klijenata za pojedinu partiju
# party_labels = df2['OZNAKA_PARTIJE'].unique().tolist() 
mask = gb[df['OZNAKA_PARTIJE']] == 1 # gb[df2['OZNAKA_PARTIJE']] vrati za svaku oznaku partije (svaki redak tablice) njenu vrijednost i usporedi je s 1
df = df[ mask.values ] # mask je neki objekt koji moram ovako koristiti, rezultat je skup gdje za svaku partiju ima jedan klijent
print('Postotak originalnog dataset-a:')
print(df.shape[0]/old_shape[0])

Postotak originalnog dataset-a:
0.9988667707530188


In [4]:
df.head()

DATUM_IZVJESTAVANJA  KLIJENT_ID  OZNAKA_PARTIJE DATUM_OTVARANJA  \
0          2013-12-31      412140         7146814      2012-08-27   
1          2013-12-31      772139         5521579      2010-06-11   
2          2013-12-31      481985         7443823      2013-01-23   
3          2013-12-31      139014         6511639      2011-09-16   
4          2013-12-31      952566         6143123      2011-05-23   

  PLANIRANI_DATUM_ZATVARANJA DATUM_ZATVARANJA  UGOVORENI_IZNOS  \
0                 2018-04-30              NaT        629155.97   
1                 2017-06-30              NaT         46000.00   
2                 2018-02-28              NaT         44679.50   
3                 2016-09-30              NaT         29959.15   
4                 2014-05-31              NaT        100000.00   

   STANJE_NA_KRAJU_PRETH_KVARTALA  STANJE_NA_KRAJU_KVARTALA  VALUTA  \
0                       639373.81                 641562.01       2   
1                        28481.94                  26887.19       1   
2                        44908.40                  45062.09       2   
3                        19691.64                  18285.89       2   
4                        24797.93                  15685.33       1   

   VRSTA_KLIJENTA PROIZVOD VRSTA_PROIZVODA  VISINA_KAMATE TIP_KAMATE  STAROST  \
0            1420   TM0109               L           1.50          A       22   
1            1410   FL0801               A           9.70          B       38   
2            1550   FD0100               L           3.10          A       37   
3            1410   FL0801               A           7.95          B       49   
4            1410   FL0801               A           9.70          B       35   

  PRIJEVREMENI_RASKID  
0                   N  
1                   N  
2                   N  
3                   N  
4                   N

In [5]:
df.describe()

KLIJENT_ID  OZNAKA_PARTIJE  UGOVORENI_IZNOS  \
count  5.187239e+06    5.187239e+06     5.187239e+06   
mean   7.206729e+05    7.153406e+06     2.536863e+05   
std    1.161272e+07    8.757979e+06     3.768251e+06   
min    1.002000e+03    1.630070e+05     0.000000e+00   
25%    3.708930e+05    4.276645e+06     2.953413e+04   
50%    6.930060e+05    6.605459e+06     6.414009e+04   
75%    1.032869e+06    9.143072e+06     1.150000e+05   
max    9.991411e+09    1.136257e+08     6.055048e+08   

       STANJE_NA_KRAJU_PRETH_KVARTALA  STANJE_NA_KRAJU_KVARTALA        VALUTA  \
count                    4.766322e+06              5.187239e+06  5.187239e+06   
mean                     1.295566e+05              1.168920e+05  1.682218e+00   
std                      1.920337e+06              1.801749e+06  8.217323e-01   
min                     -5.981000e+01             -5.981000e+01  1.000000e+00   
25%                      6.878780e+03              3.977950e+02  1.000000e+00   
50%                      3.335964e+04              2.670734e+04  2.000000e+00   
75%                      8.274797e+04              7.624641e+04  2.000000e+00   
max                      6.105662e+08              6.105662e+08  5.000000e+00   

       VRSTA_KLIJENTA  VISINA_KAMATE       STAROST  
count    5.187239e+06   5.135555e+06  5.187239e+06  
mean     1.426424e+03   5.561989e+00  4.140727e+01  
std      6.022722e+01   3.760708e+00  2.046405e+02  
min      1.110000e+03  -4.000000e-03 -7.994000e+03  
25%      1.410000e+03   2.350000e+00  3.500000e+01  
50%      1.410000e+03   6.480000e+00  4.600000e+01  
75%      1.410000e+03   8.740000e+00  5.700000e+01  
max      1.610000e+03   8.080000e+01  9.080000e+02

In [6]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                     0
KLIJENT_ID                              0
OZNAKA_PARTIJE                          0
DATUM_OTVARANJA                         0
PLANIRANI_DATUM_ZATVARANJA         420978
DATUM_ZATVARANJA                  3973234
UGOVORENI_IZNOS                         0
STANJE_NA_KRAJU_PRETH_KVARTALA     420917
STANJE_NA_KRAJU_KVARTALA                0
VALUTA                                  0
VRSTA_KLIJENTA                          0
PROIZVOD                                0
VRSTA_PROIZVODA                         0
VISINA_KAMATE                       51684
TIP_KAMATE                              0
STAROST                                 0
PRIJEVREMENI_RASKID                     0
dtype: int64

Varijablom 'STAROST' ćemo se kasnije pozabaviti pa trenutačno nelogične vrijednosti postavljamo na NaN, kako bismo ih kasnije mogli lakše identificirati i popuniti.

In [7]:
df['STAROST'] = np.where((df['STAROST']<0) | (df['STAROST']>120), np.nan, df['STAROST'])

In [8]:
## SORTIRANJE za lakši pregled
df.sort_values(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA', 'DATUM_IZVJESTAVANJA'], inplace = True)

Odlučili smo da nećemo koristiti podatke o stanju kvartala. Prvotna ideja je bila probati koristiti metode takozvanog privilegiranog učenja, no nije nam ostalo dovoljno vremena.

In [9]:
del_col_list = ['STANJE_NA_KRAJU_PRETH_KVARTALA', 'STANJE_NA_KRAJU_KVARTALA']
df.drop(del_col_list, axis = 1, inplace = True)

U sljedećoj ćeliji se popunjavaju nedostajuće vrijednosti. Neke vrijednosti se nalaze tek u kasnijim izvještajima pa ćemo ih dobiti pomoću groupby objekta i funkcije 'transform'.

Ponegdje je ugovoreni iznos ugovora jednak 0 samo u prvom izvještaju pa također mijenjamo tu varijablu. 

Ugovore koji su se produljivali ovdje smatramo kao jedan ugovor. 

In [12]:
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'])
grouped_all = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])

def impute_backwards(series):
    return series.fillna(method = 'bfill')

def impute_mean(series):
    return series.fillna(series.mean())

def impute_max(group):
    return group.max()

df['VISINA_KAMATE'] = grouped['VISINA_KAMATE'].transform(impute_mean)
df['PLANIRANI_DATUM_ZATVARANJA']  = grouped_all['PLANIRANI_DATUM_ZATVARANJA'].transform('last')
df['UGOVORENI_IZNOS'] = grouped['UGOVORENI_IZNOS'].transform('last') #ili max
df['DATUM_ZATVARANJA'] = grouped_all['DATUM_ZATVARANJA'].transform('last')
df['PRIJEVREMENI_RASKID'] = grouped_all['PRIJEVREMENI_RASKID'].transform('last')
df['DATUM_OTVARANJA'] = grouped_all['DATUM_OTVARANJA'].transform('first')

In [14]:
### DISTRIBUCIJA CILJNE ZNAČAJKE NAKON PRVE TRANSFORMACIJE
duplikati = df.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE'])
(duplikati.groupby(['PRIJEVREMENI_RASKID']).size()) /(duplikati.shape[0])

RASKID
N    0.788627
Y    0.211373
dtype: float64

In [15]:
df.isnull().sum()

DATUM_IZVJESTAVANJA                 0
KLIJENT_ID                          0
OZNAKA_PARTIJE                      0
DATUM_OTVARANJA                     0
PLANIRANI_DATUM_ZATVARANJA      14836
DATUM_ZATVARANJA              1306549
UGOVORENI_IZNOS                     0
VALUTA                              0
VRSTA_KLIJENTA                      0
PROIZVOD                            0
VRSTA_PROIZVODA                     0
VISINA_KAMATE                    4668
TIP_KAMATE                          0
STAROST                          3834
PRIJEVREMENI_RASKID                 0
RASKID                              0
dtype: int64

Brišemo ugovore koji još nisu zatvoreni, a i one kojima je nepoznata značajka 'PLANIRANI_DATUM_ZATVARANJA' jer je planiramo koristiti u definiciji nove značajke.

In [17]:
df.dropna(subset = ['DATUM_ZATVARANJA'], inplace = True)
df.dropna(subset = ['PLANIRANI_DATUM_ZATVARANJA'], inplace = True)

In [18]:
### NAPOMENA S FAQ-a
df['PRIJEVREMENI_RASKID'] = np.where(df['DATUM_ZATVARANJA']>=(df['PLANIRANI_DATUM_ZATVARANJA']- np.timedelta64(10, 'D')), 
                                     'N', df['PRIJEVREMENI_RASKID'])

In [19]:
df.isnull().sum()

DATUM_IZVJESTAVANJA              0
KLIJENT_ID                       0
OZNAKA_PARTIJE                   0
DATUM_OTVARANJA                  0
PLANIRANI_DATUM_ZATVARANJA       0
DATUM_ZATVARANJA                 0
UGOVORENI_IZNOS                  0
VALUTA                           0
VRSTA_KLIJENTA                   0
PROIZVOD                         0
VRSTA_PROIZVODA                  0
VISINA_KAMATE                 3952
TIP_KAMATE                       0
STAROST                       1747
PRIJEVREMENI_RASKID              0
RASKID                           0
dtype: int64

In [20]:
df.shape

(3872955, 16)

In [21]:
# brisanje duplikata
df.drop_duplicates(subset = ['KLIJENT_ID', 'OZNAKA_PARTIJE', 'DATUM_OTVARANJA'], inplace = True) # keep first - default value
# distribucija raskida
df.groupby(['PRIJEVREMENI_RASKID']).size()/(df.shape[0])

RASKID
N    0.741685
Y    0.258315
dtype: float64

Definiramo nove značajke; godinu otvaranja i planiranu duljinu trajanja kredita (nakon svih produljivanja).

In [23]:
df['GODINA_OTVARANJA'] = df['DATUM_OTVARANJA'].dt.year
df['PLANIRANA_DULJINA_KREDITA'] = (df['PLANIRANI_DATUM_ZATVARANJA']-df['DATUM_OTVARANJA']).dt.days
# df['DULJINA_KREDITA'] =  (df['DATUM_ZATVARANJA']-df['DATUM_OTVARANJA']).dt.days # toga nema u test-setu pa necemo koristiti

In [24]:
df.isnull().sum()

DATUM_IZVJESTAVANJA              0
KLIJENT_ID                       0
OZNAKA_PARTIJE                   0
DATUM_OTVARANJA                  0
PLANIRANI_DATUM_ZATVARANJA       0
DATUM_ZATVARANJA                 0
UGOVORENI_IZNOS                  0
VALUTA                           0
VRSTA_KLIJENTA                   0
PROIZVOD                         0
VRSTA_PROIZVODA                  0
VISINA_KAMATE                 1320
TIP_KAMATE                       0
STAROST                        248
PRIJEVREMENI_RASKID              0
GODINA_OTVARANJA                 0
PLANIRANA_DULJINA_KREDITA        0
dtype: int64

In [25]:
df.shape

(473809, 17)

Popunjavamo zadnje nedostajuće vrijednosti; varijable 'STAROST' i 'VISINA_KAMATE'.

In [26]:
grouped_kamate= df.groupby(['VRSTA_KLIJENTA', 'VRSTA_PROIZVODA'])
grouped_klijenti = df.groupby(['VRSTA_KLIJENTA'])
grouped =  df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])

def impute_mean(series):
    return series.fillna(series.mean())

df['VISINA_KAMATE'] = grouped_kamate.transform(impute_mean) # mozda oni uzimaju prve instance samo
df['STAROST'] = grouped_klijenti.transform(impute_mean)

Definiramo novu značajku - varijablu 'PRODULJENJA' - koja nam kazuje koliko puta je ugovor produžen (računa se i prvo otvaranje).

In [27]:
grouped = df.groupby(['KLIJENT_ID', 'OZNAKA_PARTIJE'])
df['PRODULJENJA'] = grouped['KLIJENT_ID'].transform('size') # svejedno po kojoj instanci gledam

In [30]:
df.isnull().sum()

DATUM_IZVJESTAVANJA           0
KLIJENT_ID                    0
OZNAKA_PARTIJE                0
DATUM_OTVARANJA               0
PLANIRANI_DATUM_ZATVARANJA    0
DATUM_ZATVARANJA              0
UGOVORENI_IZNOS               0
VALUTA                        0
VRSTA_KLIJENTA                0
PROIZVOD                      0
VRSTA_PROIZVODA               0
VISINA_KAMATE                 0
TIP_KAMATE                    0
STAROST                       0
PRIJEVREMENI_RASKID           0
GODINA_OTVARANJA              0
PLANIRANA_DULJINA_KREDITA     0
PRODULJENJA                   0
dtype: int64

In [31]:
df.shape

(473809, 18)

In [39]:
df.columns

Index(['DATUM_IZVJESTAVANJA', 'KLIJENT_ID', 'OZNAKA_PARTIJE',
       'DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_ZATVARANJA',
       'UGOVORENI_IZNOS', 'VALUTA', 'VRSTA_KLIJENTA', 'PROIZVOD',
       'VRSTA_PROIZVODA', 'VISINA_KAMATE', 'TIP_KAMATE', 'STAROST',
       'PRIJEVREMENI_RASKID', 'GODINA_OTVARANJA', 'PLANIRANA_DULJINA_KREDITA',
       'PRODULJENJA'],
      dtype='object')

Napokon, brišemo varijable koje nam više ne trebaju te eksportiramo podatke u novu datoteku.

In [40]:
del_col_list = ['DATUM_IZVJESTAVANJA', 'DATUM_OTVARANJA', 'PLANIRANI_DATUM_ZATVARANJA', 'DATUM_ZATVARANJA']
df.drop(del_col_list, axis = 1, inplace = True)

In [42]:
df.to_hdf('train_dataset.h5', key='df')